In [ ]:
""" Quel périmetre?

- visite du site:
    - trustpilot filtre sur le pays (truspilot US, truspilot FR etc.)
    - catégories -> 22 (aliment, élétronique, santé etc.)
    - sous-catégories -> une dizaine max par catégories
        - pages catégorie -> nombre d'avis, score, le lieu, nom entreprise, sous-cat (+)
        - pages catégorie -> trier par avis pertinent, avec le plus d'avis, avis les plus recent
            - pages entreprise -> (+) titre de l'avis, le texte de l'avis, le lieu de l'avis, la date, le username, nombre davis effectué par la personne
            - pages entreprise -> réponse de l'entreprise, date
                - pages personne -> tous ces avis toute entreprise confondue

- on peut commencer par prendre le site Truspilote en francais -> les entreprises en france
- toute la france
- quelles catégories?
    - 
- problématique: 
    - quelles entreprises sont les mieux notées par catégories (banques, web etc.), les tendances par années
    - peut etre catégorie aliment + 
"""

In [ ]:
"""
Quelle solution pour récuperer de la données?:
- Scrapping
- API truspilot doc ici https://documentation-apidocumentation.trustpilot.com/
    - nécessite un compte business (demande validation que le site web exite vraiment)
"""

In [34]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
#import pandas as pd

In [75]:
# https://fr.trustpilot.com/categories/electronics_technology

def displaySrapData(url_sub_category = "https://fr.trustpilot.com/categories/electronics_technology"):
    
    tpPage = urlopen(url_sub_category)
    soup = BeautifulSoup(tpPage, 'html.parser')

    tpTitles = soup.findAll(name = 'p', attrs = {'class': 'styles_displayName__1LIcI'})
    tpMetrics = soup.findAll(name = 'p', attrs = {'class': 'styles_ratingText__nheL7'})
    tpPlaces = soup.findAll(name = 'p', attrs = {'class': 'styles_location__3JATO'})
    tpSubCategories = soup.findAll(name = 'div', attrs = {'class': 'styles_desktop__3N0-b'})
    tpWebsite = soup.findAll(name = 'a', href = True, attrs = {'class': 'styles_linkWrapper__3x9X7'})


    title = [] # On crée une liste vide qui contiendra tous les titres propres
    for element in tpTitles:
        title.append(element.text.encode('ascii', errors='ignore').decode('utf-8'))

    rating = []
    review = []
    for element in tpMetrics:
        rating.append(element.text.encode('ascii', errors='ignore').decode('utf-8').split()[1].split("|")[0])
        review.append(element.text.encode('ascii', errors='ignore').decode('utf-8').split()[1].split("|")[1])

    city = []
    country = []
    for element in tpPlaces:
        city.append(element.text.encode('ascii', errors='ignore').decode('utf-8').split(",")[0])
        country.append(element.text.encode('ascii', errors='ignore').decode('utf-8').split(",")[1])

    subCategory = []
    for element in tpSubCategories:
        subCategory.append(element.text.encode('ascii', errors='ignore').decode('utf-8'))

    website = []
    for element in tpWebsite:
        website.append(element["href"])




In [76]:
print(website)

['/review/www.cleanfox.io', '/review/fonebank.fr', '/review/affinitweet.com', '/review/spliiit.com', '/review/wp-rocket.me', '/review/gamefix.fr', '/review/transfernow.net', '/review/www.europe-nature-optik.fr', '/review/sonner.tel', '/review/greenvallee.fr', '/review/www.docorga.com', '/review/www.francechargeur.fr', '/review/bmjelec.com', '/review/sga-reparation.fr', '/review/techpower.fr', '/review/ordipack.fr', '/review/pc-gamer.tech', '/review/hedee-store.com', '/review/epycore.fr', '/review/mshop.fr']
